## 1. Entering variables

In [ ]:
import json
import requests
import datetime
import sqlite3
import pandas as pd
import getpass

tickers = ['XOM', 'CVX', 'COP', 'EOD', 'EPD', 'MPC', 'PSX', 'OXY', 'ET', 'WMB',
           'VLO', 'OKE']
api_key = getpass.getpass('Enter your API key: ')
# availiable candle size: 1 (1 minute), 5 (5 minutes), 15 (1 minutes), 30 (30 minutes), 60 (60 minutes), 24 (daily)
candle_size = 24
db_path = "/content/drive/MyDrive/databases/candles_history.db"

Enter your API key: ··········


## 2. Receiving JSON candles and save to Database

In [ ]:
for ticker in tickers:
  data = get_daily_candles(ticker, api_key)
  candles = candles_processing(data, ticker, candle_size)
  candles_save(candles, db_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/conte

## Candle model class

In [ ]:
class Candle():
  @property
  def id(self):
    pass

  @property
  def date_time(self):
    pass

  @property
  def ticker(self):
    pass

  @property
  def size(self):
    pass

  @property
  def source(self):
    pass

  @property
  def open(self):
    pass

  @property
  def max(self):
    pass

  @property
  def min(self):
    pass

  @property
  def close(self):
    pass

  @property
  def volume(self):
    pass

  def __init__(self, id: str, date_time: datetime.datetime, ticker: str, size: int,
            source: str, open: float, max: float, min: float, close: float,
            volume: int):
    self._id = id
    self._date_time = date_time
    self._ticker = ticker
    self._size = size
    self._source = source
    self._open = open
    self._max = max
    self._min = min
    self._close = close
    self._volume = volume

  def __str__(self) -> str:
    return f"datetime: {self._date_time}, open: {self._open}, high: {self._max}, low: {self._min}, close: {self._close}, volume: {self._volume}"

  def __repr__(self) -> str:
    return f"datetime: {self._date_time}, open: {self._open}, high: {self._max}, low: {self._min}, close: {self._close}, volume: {self._volume}"

## Function for processing received candles JSON

In [ ]:
def candles_processing(data, ticker, candle_size):
  candle_keys = [key for key in data['Time Series (Daily)']]
  candles = []
  source = 'global'
  for key in candle_keys:
    id = str(key) + str(ticker) + str(candle_size)
    candle = Candle(id,
                    datetime.datetime.strptime(key, '%Y-%m-%d'),
                    ticker,
                    candle_size,
                    source,
                    float(data['Time Series (Daily)'][key]['1. open']),
                    float(data['Time Series (Daily)'][key]['2. high']),
                    float(data['Time Series (Daily)'][key]['3. low']),
                    float(data['Time Series (Daily)'][key]['4. close']),
                    int(data['Time Series (Daily)'][key]['5. volume']))
    candles.append(candle)
  return candles

## Candles request functions

In [ ]:
def get_daily_candles(ticker, api_key):
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={api_key}'
  r = requests.get(url)
  data = r.json()
  return data

## Database saving function

In [ ]:
def candles_save(candles, path):
  connect_to_drive()
  conn = sqlite3.connect(path)
  cursor = conn.cursor()

  cursor.execute('''
    CREATE TABLE IF NOT EXISTS candles (
    id TEXT PRIMARY KEY,
    date_time DATETIME,
    ticker TEXT,
    size INTEGER,
    source TEXT,
    open REAL,
    max REAL,
    min REAL,
    close REAL,
    volume INTEGER
    )
    ''')

  candles_to_insert = [(candle._id, candle._date_time, candle._ticker,
                        candle._size, candle._source, candle._open,
                        candle._max, candle._min, candle._close, candle._volume) for candle in candles]
  try:
    cursor.executemany('''
      INSERT OR IGNORE INTO candles (id, date_time, ticker, size, source, open, max, min, close, volume)
      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
      ''', candles_to_insert)
    conn.commit()
    conn.close()
  except sqlite3.Error as e:
    print(f"Error: {e}")
  finally:
        conn.close()

## Conntect to Drive

In [ ]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/drive')